# 第10章 深層学習によって実現できる画像処理・言語処理を知ろう(5-7節)
ここでは画像のセグメンテーションについて学んでいきます。

Google Colaboratory上で実行する場合、ランタイムがGPUになっていることを確認して下さい

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter10

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements2.txt

## 10-6 Segnetでセグメンテーションしてみよう

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, UpSampling2D,ZeroPadding2D, Input, BatchNormalization, Dense, Reshape

#SegNet Basicモデルの構築
def segnet(n_classes, input_shape=(224,224,3)):

    kernel = 3
    pad = 1
    pool_size = 2

    model = Sequential()
    model.add(Input(shape=input_shape))

    model.add(ZeroPadding2D((pad, pad)))
    model.add(Conv2D(64, (kernel, kernel), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

    model.add(ZeroPadding2D((pad, pad)))
    model.add(Conv2D(128, (kernel, kernel), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

    model.add(ZeroPadding2D((pad, pad)))
    model.add(Conv2D(256, (kernel, kernel), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), padding='valid'))
    model.add(BatchNormalization())

    model.add(UpSampling2D((2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), padding='valid'))
    model.add(BatchNormalization())

    model.add(UpSampling2D((2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), padding='valid'))
    model.add(BatchNormalization())

    model.add(UpSampling2D((2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(n_classes, (1, 1), padding='valid'))
    model.add(Reshape((input_shape[0] * input_shape[1], n_classes)))

    model.add(Activation("softmax"))
    model.compile(optimizer="adadelta", loss="categorical_crossentropy", metrics=['acc'])

    return model

In [ ]:
from PIL import Image

Image.open("./VOCdevkit/VOC2007/JPEGImages/000793.jpg")

In [ ]:
label_image = Image.open("./VOCdevkit/VOC2007/SegmentationClass/000793.png")
label_image

In [ ]:
import numpy as np
label_image = np.array(label_image)

#ラベルデータの９０行目を表示
label_image[90]

In [ ]:
from tensorflow.keras.utils import Sequence
import numpy as np
import math 
from PIL import Image

#画像のクロップ
def crop_center(pil_img, crop_height, crop_width):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

def normalized(img):
  norm = img/255
  return norm

def one_hot(labels, data_shape, class_num):
  x = np.zeros([data_shape[0],data_shape[1], class_num])
  for i in range(data_shape[0]):
      for j in range(data_shape[1]):
        if labels[i][j]  == 0 or labels[i][j] == 255:
          x[i,j,0] = 1
        else:
          x[i,j,labels[i][j]] = 1
        
  return x

In [ ]:
#学習時に画像データを必要な分だけ読み込むためのクラス
class ImageDataSequence(Sequence):
    def __init__(self, data_shape, class_num, image_file_name_list, batch_size):
        
        self.file_name_list = image_file_name_list

        self.image_file_name_list = ["./VOCdevkit/VOC2007/JPEGImages/"+image_path + ".jpg" for image_path in self.file_name_list]
        self.label_image_file_name_list = ['./VOCdevkit/VOC2007/SegmentationClass/' + image_path+ ".png" for image_path in self.file_name_list]

        self.length = len(self.file_name_list)
        self.data_shape = data_shape
        self.class_num = class_num
        self.batch_size = batch_size

    def __getitem__(self, idx):
        images = []
        label_images = []
        for index in range(idx*self.batch_size, (idx+1)*self.batch_size):
          img = Image.open(self.image_file_name_list[index])
          img = crop_center(img,self.data_shape[0],self.data_shape[1]  )
          img = normalized(np.array(img))
          images.append(img)

          label_img = Image.open(self.label_image_file_name_list[index])
          label_img = crop_center(label_img, self.data_shape[0],self.data_shape[1])
          label_img = one_hot(np.array(label_img), self.data_shape, self.class_num)
          label_img = label_img.reshape(self.data_shape[0]*self.data_shape[1], self.class_num)
          label_images.append(label_img)
          
        return np.array(images), np.array(label_images)

    def __len__(self):
        return math.floor(len(self.file_name_list) / self.batch_size)


## 10-7 セグメンテーションした結果を評価してみよう

In [ ]:
train_file = open("./VOCdevkit/VOC2007/ImageSets/Segmentation/train.txt")
test_file = open("./VOCdevkit/VOC2007/ImageSets/Segmentation/val.txt")
train_file_names = train_file.read().split("\n")
test_file_names = test_file.read().split("\n")

train_file_names.pop(-1)
test_file_names.pop(-1)

#テストデータの内５０個を検証用に使う
val_file_names = test_file_names[0:50]
test_file_names = test_file_names[50:]


input_shape =(400,400, 3)

#0=non
#1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle, 6=bus, 7=car , 8=cat, 9=chair, 10=cow, 
#11=diningtable, 12=dog, 13=horse, 14=motorbike, 15=person, 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
n_labels =  21


#モデルの構築
model = segnet(n_labels,input_shape=input_shape )

#モデルの構造の表示
model.summary()

#データを読み込む為のインスタンス作成
train_gen = ImageDataSequence(input_shape, n_labels, train_file_names, 8)
val_gen = ImageDataSequence(input_shape, n_labels, val_file_names, 8)
test_gen = ImageDataSequence(input_shape, n_labels, test_file_names, 8)

In [ ]:

#学習開始
history = model.fit(
    train_gen,
    epochs=40,
    steps_per_epoch=len(train_gen),
    validation_data=val_gen,
    verbose=1)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='g', label="Validation Loss")
legend = ax[0].legend()

ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_acc'], color='g', label="Validation Accuracy")
legend = ax[1].legend()

In [ ]:
#学習に時間がかかるため、以下のコードのコメントアウトを外すことでモデルの重みの保存と読み込みが出来ます.
#１度保存しておき、8節以降をあとから実行する場合保存したモデルの重みを読み込むことで学習を再実行しなくて済みます

#学習した重みの保存
#model.save_weights('./saved_models/model_segnet_weights')

In [ ]:
#重みの読み込み
#model = segnet(n_labels,input_shape=input_shape )
#model.load_weights('./saved_models/model_segnet_weights')

In [ ]:
#学習したモデルでセグメンテーションを行う

#画像の読み込みと前処理
tmp_name = "./VOCdevkit/VOC2007/JPEGImages/" + test_file_names[2] + ".jpg"
img = Image.open(tmp_name)
img = crop_center(img,input_shape[0],input_shape[1]  )
img = normalized(np.array(img))

#元の画像を表示
plt.imshow(img)
plt.show()

#推論開始
result = model.predict(np.array([img]))

#出力の配列を加工
_result = result.reshape((400,400,21)).argmax(axis=2)

#セグメンテーション結果の表示
plt.imshow(_result)
plt.show()